In [1]:
import os
import re
import time
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta


# ігнорування попередження SettingWithCopyWarning
pd.set_option('mode.chained_assignment', None)  


In [2]:
list_columns =   [
        "Date",
        "HomeTeam",
        "AwayTeam",
        "rez_h_full",
        "rez_a_full",
        "rez_h_first",
        "rez_a_first",
        "h_first",
        "h_full",
        "a_first",
        "a_full",
    ] 
# funcs

def get_team_results(
                    h_team_name: str,
                    a_team_name: str,
                    matches_before: pd.DataFrame,
                    ) -> tuple[int, int, int, int] :
    """
    отримання результатів команд, останнього матчу
    повертає список з 4 чисел, різниці забитих мячів
      - дом. команди в першому таймі,
      - дом. команди в повному матчі,
      - гост. команди в першому таймі,
      - гост. команди в повному матчі,
    """
    h_team_match = matches_before[                            
                        (matches_before["HomeTeam"] == h_team_name) | 
                        (matches_before["AwayTeam"] == h_team_name)
                    ].tail(1) 
    
    a_team_match = matches_before[
                        (matches_before["HomeTeam"] == a_team_name) |
                        (matches_before["AwayTeam"] == a_team_name)
                    ].tail(1) 
    
#     print()
#     print(h_team_match)
#     print()
#     print(a_team_match)
#     print()

    h_first = (
                    h_team_match.at[h_team_match.index[0], "HTHG"] - 
                    h_team_match.at[h_team_match.index[0], "HTAG"]
                )


    h_full = (
                    h_team_match.at[h_team_match.index[0], "FTHG"] - 
                    h_team_match.at[h_team_match.index[0], "FTAG"]
                )

    h_first = h_first if h_team_name in h_team_match["HomeTeam"].values else -h_first
    h_full = h_full if h_team_name in h_team_match["HomeTeam"].values else -h_full


    a_first = (
                    a_team_match.at[a_team_match.index[0], "HTHG"] - 
                    a_team_match.at[a_team_match.index[0], "HTAG"]
                )

    a_full = (
                    a_team_match.at[a_team_match.index[0], "FTHG"] - 
                    a_team_match.at[a_team_match.index[0], "FTAG"]
                )
    a_first = a_first if a_team_name in a_team_match["HomeTeam"].values else -a_first
    a_full = a_full if a_team_name in a_team_match["HomeTeam"].values else -a_full
    
    return (
        h_first,
        h_full,
        a_first,
        a_full,
    )

In [6]:
# конвертування строчки дати в формат дати 
d, m, y = [int(x) for x in df.at[0, "Date"].split("/")]
a = date(y, m, d)
a
# date(2023, 5, 9).isoweekday() == 2  # первірка дати на вівторок

AttributeError: 'datetime.date' object has no attribute 'split'

In [3]:
df = pd.read_csv("./data/eo_21_22.csv")
df.loc[:, "Date"] =  [date.fromisoformat("-".join(x.split("/")[::-1]))  for x in df.loc[:, "Date"].values]

df

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,2021-08-13,20:00,Brentford,Arsenal,2,0,H,1,0,...,1.62,0.50,1.75,2.05,1.81,2.13,2.05,2.17,1.80,2.09
1,E0,2021-08-14,12:30,Man United,Leeds,5,1,H,1,0,...,2.25,-1.00,2.05,1.75,2.17,1.77,2.19,1.93,2.10,1.79
2,E0,2021-08-14,15:00,Burnley,Brighton,1,2,A,1,0,...,1.62,0.25,1.79,2.15,1.81,2.14,1.82,2.19,1.79,2.12
3,E0,2021-08-14,15:00,Chelsea,Crystal Palace,3,0,H,2,0,...,1.94,-1.50,2.05,1.75,2.12,1.81,2.16,1.93,2.06,1.82
4,E0,2021-08-14,15:00,Everton,Southampton,3,1,H,0,1,...,1.67,-0.50,2.05,1.88,2.05,1.88,2.08,1.90,2.03,1.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,2022-05-22,16:00,Crystal Palace,Man United,1,0,H,1,0,...,2.04,0.25,1.68,2.15,1.74,2.23,1.88,2.25,1.74,2.16
376,E0,2022-05-22,16:00,Leicester,Southampton,4,1,H,0,0,...,2.63,-0.75,1.83,2.07,1.88,2.03,1.94,2.26,1.87,2.01
377,E0,2022-05-22,16:00,Liverpool,Wolves,3,1,H,1,1,...,3.28,-2.50,2.02,1.77,2.06,1.83,2.19,1.99,2.07,1.80
378,E0,2022-05-22,16:00,Man City,Aston Villa,3,2,H,0,1,...,3.36,-2.25,2.06,1.84,2.05,1.86,2.09,2.03,2.01,1.87


In [7]:
df_data = df.loc[:, ["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "HTHG", "HTAG"]]
df_data

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HTHG,HTAG
0,2021-08-13,Brentford,Arsenal,2,0,1,0
1,2021-08-14,Man United,Leeds,5,1,1,0
2,2021-08-14,Burnley,Brighton,1,2,1,0
3,2021-08-14,Chelsea,Crystal Palace,3,0,2,0
4,2021-08-14,Everton,Southampton,3,1,0,1
...,...,...,...,...,...,...,...
375,2022-05-22,Crystal Palace,Man United,1,0,1,0
376,2022-05-22,Leicester,Southampton,4,1,0,0
377,2022-05-22,Liverpool,Wolves,3,1,1,1
378,2022-05-22,Man City,Aston Villa,3,2,0,1


In [8]:
rez_data = []
for ind in df_data.index:
    current_match = df_data.loc[ind]
    matches_before = df_data.loc[:ind-1]
    h_team_name = current_match["HomeTeam"]
    a_team_name = current_match["AwayTeam"]
    if ( 
        (h_team_name in matches_before["HomeTeam"].values or
                 h_team_name in matches_before["AwayTeam"].values) and 
        (a_team_name in matches_before["HomeTeam"].values or
                 a_team_name in matches_before["AwayTeam"].values)  
    ):
        h_first, h_full, a_first, a_full = get_team_results(
                                                     h_team_name,
                                                     a_team_name,
                                                    matches_before
                                                    )
        
        print(ind, h_team_name, a_team_name)
        print()
        print(matches_before.tail(15))
        print()
        
        print(h_full, h_first)
        
        print()
        print(a_full, a_first)
        
        data_row = [
            *current_match.values,
            h_first,
            h_full,
            a_first,
            a_full
        ]
#         print(data_row)
        rez_data.append(data_row)
#         raise

10 Liverpool Burnley

         Date    HomeTeam        AwayTeam  FTHG  FTAG  HTHG  HTAG
0  2021-08-13   Brentford         Arsenal     2     0     1     0
1  2021-08-14  Man United           Leeds     5     1     1     0
2  2021-08-14     Burnley        Brighton     1     2     1     0
3  2021-08-14     Chelsea  Crystal Palace     3     0     2     0
4  2021-08-14     Everton     Southampton     3     1     0     1
5  2021-08-14   Leicester          Wolves     1     0     1     0
6  2021-08-14     Watford     Aston Villa     3     2     2     0
7  2021-08-14     Norwich       Liverpool     0     3     0     1
8  2021-08-15   Newcastle        West Ham     2     4     2     1
9  2021-08-15   Tottenham        Man City     1     0     0     0

3 1

-1 1
11 Aston Villa Newcastle

          Date    HomeTeam        AwayTeam  FTHG  FTAG  HTHG  HTAG
0   2021-08-13   Brentford         Arsenal     2     0     1     0
1   2021-08-14  Man United           Leeds     5     1     1     0
2   2021-08-14

236 Aston Villa Watford

           Date     HomeTeam        AwayTeam  FTHG  FTAG  HTHG  HTAG
221  2022-02-09  Aston Villa           Leeds     3     3     3     2
222  2022-02-10    Liverpool       Leicester     2     0     1     0
223  2022-02-10       Wolves         Arsenal     0     1     0     1
224  2022-02-12   Man United     Southampton     1     1     1     0
225  2022-02-12    Brentford  Crystal Palace     0     0     0     0
226  2022-02-12      Everton           Leeds     3     0     2     0
227  2022-02-12      Watford        Brighton     0     2     0     1
228  2022-02-12      Norwich        Man City     0     4     0     1
229  2022-02-13      Burnley       Liverpool     0     1     0     1
230  2022-02-13    Newcastle     Aston Villa     1     0     1     0
231  2022-02-13    Tottenham          Wolves     0     2     0     2
232  2022-02-13    Leicester        West Ham     2     2     1     1
233  2022-02-15   Man United        Brighton     2     0     0     0
234  2022

In [9]:
clean_data = pd.DataFrame(
                        data=rez_data,
                        columns=list_columns
            )
clean_data.head(25)

,Date,HomeTeam,AwayTeam,rez_h_full,rez_a_full,rez_h_first,rez_a_first,h_first,h_full,a_first,a_full
0,2021-08-21,Liverpool,Burnley,2,0,1,0,1,3,1,-1
1,2021-08-21,Aston Villa,Newcastle,2,0,1,0,-2,-1,1,-2
2,2021-08-21,Crystal Palace,Brentford,0,0,0,0,-2,-3,1,2
3,2021-08-21,Leeds,Everton,2,2,1,1,-1,-4,-1,2
4,2021-08-21,Man City,Norwich,5,0,2,0,0,-1,-1,-3
5,2021-08-21,Brighton,Watford,2,0,2,0,-1,1,2,1
6,2021-08-22,Southampton,Man United,1,1,1,0,1,-2,1,4
7,2021-08-22,Wolves,Tottenham,0,1,0,1,-1,-1,0,1
8,2021-08-22,Arsenal,Chelsea,0,2,0,2,-1,-2,2,3
9,2021-08-23,West Ham,Leicester,4,1,1,0,-1,2,1,1


In [51]:
start_day = clean_data["Date"].values[0]
end_day = clean_data["Date"].values[-1]

list_days = []
while start_day <= end_day:
    list_days.append(start_day)
    start_day += timedelta(days=1)
    
only_tuesdays = list(
                    filter(
                        lambda day: day.isoweekday() == 2,
                        list_days
                    )
                )

# only_tuesdays

In [117]:
for tuesday in only_tuesdays:
    some_rez = np.array([], dtype=int).reshape(0,9)
    
    work_df = clean_data[
        (clean_data["Date"] >= tuesday - timedelta(days=14)) &
        (clean_data["Date"] < tuesday) 
    ] 
    
     
    target_df = clean_data[
        (clean_data["Date"] >= tuesday)  &
        (clean_data["Date"] < tuesday + timedelta(days=7)) 
    ]
    
    # минулі результати
    work_df["index"] = work_df.index
    work_df_np = work_df[[
                            "h_first", "h_full",
                            "a_first", "a_full",
                            "rez_h_full", "rez_a_full", "rez_h_first", "rez_a_first",
                            "index"
                ]].to_numpy()
    
   
    # майбутні результати
    target_df["index"] = target_df.index
    target_df_np = target_df[[
                            "h_first", "h_full",
                            "a_first", "a_full",
                            "rez_h_full", "rez_a_full", "rez_h_first", "rez_a_first",
                            "index"
                ]].to_numpy()
    
    
    some_range = np.arange(-5, 5, 1)
    
    # пошук в минулих результатах
    # поки не працює!!!
    for h_first in some_range:
        for h_full in some_range:
            for a_first in some_range:
                for a_full in some_range:
                    filtered_df_np = work_df_np[
                        (work_df_np[:, 0] == h_first) &
                        (work_df_np[:, 1] == h_full) &
                        (work_df_np[:, 2] == a_first) &
                        (work_df_np[:, 3] == a_full) 
                    ]
                    
#                     if filtered_df_np.shape[0] < 2 : continue
                    draw_first = filtered_df_np[
                                                filtered_df_np[:, 6] == filtered_df_np[:, 7]
                                            ]
                    tb_1_5_first = filtered_df_np[
                                                filtered_df_np[:, 6] + filtered_df_np[:, 7] > 1.5 
                                            ]
        
                    tm_1_5_first = filtered_df_np[
                                                filtered_df_np[:, 6] + filtered_df_np[:, 7] < 1.5 
                                            ]
                       
                       
#                     if draw_first.shape[0] >= 1: 
#                         some_rez = np.concatenate((some_rez,  draw_first))
                        
#                     if tb_1_5_first.shape[0] >= 1: 
#                         some_rez = np.concatenate((some_rez,  tb_1_5_first))
                        
                    if tm_1_5_first.shape[0] >= 1: 
                        some_rez = np.concatenate((some_rez,  tm_1_5_first))
    
#     if some_rez.shape[0] == 0: continue
    
    
    # пошук в майбутніх результатах
    for data in target_df_np:
        h_first = data[0]
        h_full = data[1]
        a_first = data[2]
        a_full = data[3]
 
        filtered_df_np = work_df_np[
            (work_df_np[:, 0] == h_first) &
            (work_df_np[:, 1] == h_full) &
            (work_df_np[:, 2] == a_first) &
            (work_df_np[:, 3] == a_full) 
        ]
    
        if filtered_df_np.shape[0] >= 1:
#             [print(x) for x in some_rez]
#             print(tuesday)
#             print(work_df)
#             print(target_df)
            print(data)
            print()

            print(filtered_df_np)
            print()
            print()
        
#             print("good") if  (filtered_df_np[:, 6] == filtered_df_np[:, 7]).all() else print("fuck")
#             print("good") if  (filtered_df_np[:, 6] + filtered_df_np[:, 7] > 1.5).all() else print("fuck")
#             print("good") if  (filtered_df_np[:, 6] + filtered_df_np[:, 7] < 1.5).all() else print("fuck")
#             raise
            
    
    
    


[ 1  2  0  0  3  1  0  0 29]

[[ 1  2  0  0  1  1  1  1 11]]


[ 0  1  0  0  0  0  0  0 34]

[[ 0  1  0  0  4  1  1  0 24]]


[  0   0   0   0   1   0   1   0 130]

[[  0   0   0   0   2   1   0   1 109]]


[  0   1   0   1   1   0   0   0 143]

[[  0   1   0   1   0   0   0   0 113]]


[  0   1   0  -1   7   0   3   0 149]

[[  0   1   0  -1   1   0   0   0 135]]


[  0   1  -1  -2   1   1   0   0 153]

[[  0   1  -1  -2   2   2   1   0 134]]


[ -1  -1  -1   1   2   0   2   0 164]

[[ -1  -1  -1   1   0   0   0   0 145]]


[  0   0  -1   0   2   3   0   2 177]

[[  0   0  -1   0   3   0   2   0 161]]


[  1   0   0  -1   1   1   0   0 185]

[[  1   0   0  -1   2   2   2   2 179]]


[  1   0   1   2   2   0   0   0 223]

[[  1   0   1   2   0   4   0   1 218]]


[  2   2   0   0   2   1   1   1 233]

[[  2   2   0   0   2   0   1   0 212]]


[  0  -1   0   0   0   2   0   2 240]

[[  0  -1   0   0   1   0   0   0 206]
 [  0  -1   0   0   0   2   0   1 217]]


[  1   2   1   2   0   2 

In [118]:
clean_data.loc[109]

Date           2021-11-21
HomeTeam        Tottenham
AwayTeam            Leeds
rez_h_full              2
rez_a_full              1
rez_h_first             0
rez_a_first             1
h_first                 0
h_full                  0
a_first                 0
a_full                  0
Name: 109, dtype: object

In [84]:
a = np.array([]).reshape(0,2)
# b = np.array([[2,3], [4.5]])
np.concatenate((a,  [[2,3], [4,5]]))


array([[2., 3.],
       [4., 5.]])